---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [26]:
import pandas as pd
email_df = pd.read_csv('email_network.txt', delimiter = '\t')
email_df['time'] = pd.to_datetime(email_df['time'], unit = 's')
email_df[['#Sender', 'Recipient']] = email_df[['#Sender', 'Recipient']].astype(str, inplace = True)
email_df.head() #.dtypes
#len(email_df) #82927

82927

In [31]:
def answer_one():
    
    G1 = nx.MultiDiGraph()
    for s, r, t in zip(email_df['#Sender'], email_df['Recipient'], email_df['time']):
        G1.add_edge(s, r, time = t)
    #G1 = nx.read_edgelist('email_network.txt', delimiter = '\t', nodetype = str, 
    #                      create_using = nx.MultiDiGraph(),
    #                     data = (('time', pd.to_datetime),))
    #not applying read_edgelist bcz of failure to implement datetime converting
    return G1
answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [36]:
G1 = answer_one()

In [35]:
def answer_two():
        
    G1 = answer_one()
    #print(len(G1.nodes()), len(G1.edges()))
    return (len(G1.nodes()), len(G1.edges()))

answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [37]:
def answer_three():
        
    # part 1: is_strongly_connected?
    part1 = nx.is_strongly_connected(G1)
    # part 2: is_weakly_connected?
    part2 = nx.is_weakly_connected(G1)
    
    return (part1, part2)
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [42]:
def answer_four():
        
    if nx.is_weakly_connected(G1):
        return len(G1.nodes())
    else:
        return max([len(l) for l in nx.weakly_connected_components(G1)])

answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [44]:
def answer_five():
        
    if nx.is_strongly_connected(G1):
        return len(G1.nodes())
    else:
        return max([len(l) for l in nx.strongly_connected_components(G1)])
    
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [45]:
def answer_six():
        
    G_sc = max(nx.strongly_connected_component_subgraphs(G1), key = len)
    
    return G_sc

answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [46]:
G_sc = answer_six()

In [48]:
def answer_seven():
    
    #Gsc_eccentricity = nx.eccentricity(G_sc)
    #Gsc_nodes = len(Gsc_eccentricity)
    #Gsc_total_distance = sum(Gsc_eccentricity.values())
    # Wrong, should compute shortest between nodes
    
    return nx.average_shortest_path_length(G_sc)

answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [50]:
def answer_eight():
    #return max(nx.eccentricity(G_sc).values()) #same as below
    return nx.diameter(G_sc)
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [51]:
def answer_nine():
    
    return nx.periphery(G_sc) #periphery nodes are those with the largest distance in the network
answer_nine()

['97', '134', '129']

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [52]:
def answer_ten():
    
    return nx.center(G_sc)
answer_ten()

['38']

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [59]:
#p = nx.shortest_path_length(G_sc, source = '97')
#node_num = Counter(p.values())
#node_num

Counter({0: 1, 1: 1, 2: 61, 3: 63})

In [61]:
from collections import Counter
def answer_eleven():
    peripheries = nx.periphery(G_sc)
    l_d = nx.diameter(G_sc)
    max_connected_nodes = ""
    max_node_num = 0
    for p in peripheries:
        spl = nx.shortest_path_length(G_sc, source = p)
        node_num = Counter(spl.values())
        if node_num[l_d] > max_node_num:
            max_node_num = node_num[l_d]
            max_connected_nodes = p
    
    return (max_connected_nodes, max_node_num)
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [65]:
def answer_twelve():
        
    # prevent communication from flowing to the node that you found ('97') in the previous question 
    # from any node in the center of G_sc ('38')
    s = answer_ten()[0]
    t = answer_eleven()[0]
    
    #return nx.node_connectivity(G_sc, s = s, t = t) #6
    
    return len(nx.minimum_node_cut(G_sc, s = s, t = t))

answer_twelve()

5

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [68]:
def answer_thirteen():
        
    G_un = G_sc.to_undirected()
    
    return nx.Graph(G_un)

answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [69]:
def answer_fourteen():
        
    G_un = answer_thirteen()
    avg_clu_coef = nx.average_clustering(G_un)
    transitivity = nx.transitivity(G_un)
    
    return (transitivity, avg_clu_coef)
answer_fourteen()

(0.570111160700385, 0.6975272437231417)